# "The disturbed outer Milky Way disc", McMillan et al (2022)
# Create Gaia data plots using proper motion only

### Package requirements: numpy, pandas, matplotlib, astropy, dustmaps

# Version history

v1 - Copied across from my ugly code to make public with initial arXiv publication

In [ ]:
datadir = 'BailerJones/'
plotdir = 'GaiaData/'

import os

if not os.path.exists(plotdir):
    os.makedir(plotdir)

In [ ]:
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt

# Convert proper motions to l, b (can also do uncertainties!)
import coordTransform_matrix

# Convenient set of plotting bits and pieces 
from plotparameters import *
import pandas as pd

#fig, ax = plt.subplots(figsize=(6,3.5)) # for one-column plots
#fig, ax = plt.subplots(figsize=(13,7.6)) # for double-column plots

In [ ]:


# Use plotparameters function to send plasma to white at the low value end. 
# Makes plots less yellow and ugly.
mycolormap = generatecmap('plasma_r')



The data tables are downloaded from the Gaia Archive (https://gea.esac.esa.int/archive/).
Each file covers 20 degrees, to keep file size/query time down.

SELECT source_id, ra, dec, parallax, parallax_error, pmra, pmdec, phot_g_mean_mag, bp_rp,l,b, r_med_photogeo, r_lo_photogeo, r_hi_photogeo  
FROM external.gaiaedr3_distance  
JOIN gaiaedr3.gaia_source USING (source_id)  
WHERE l<150 and l>130 and b>-10 and b<10 and parallax_over_error>=3 and parallax IS NOT NULL


Files were downloaded as csv, then converted to hdf via

table = pd.read_csv(f'{filename}.csv')  
table.to_hdf(f'{filename}.h5','table', mode='w')


In [ ]:
df_130 = pd.read_hdf(datadir + 'ACBJ130_150-result.h5', 'table')
df_150 = pd.read_hdf(datadir + 'ACBJ150_170-result.h5', 'table')
df_170 = pd.read_hdf(datadir + 'ACBJ170_190-result.h5', 'table')
df_190 = pd.read_hdf(datadir + 'ACBJ190_210-result.h5', 'table')
df_210 = pd.read_hdf(datadir + 'ACBJ210_230-result.h5', 'table')

df = pd.concat([df_130,df_150,df_170,df_190,df_210],ignore_index=True)
# convert to kpc
df['r_med_photogeo'] = df['r_med_photogeo']/1000.
df['r_lo_photogeo'] =  df['r_lo_photogeo']/1000.
df['r_hi_photogeo'] =  df['r_hi_photogeo']/1000.
df_130 = []
df_150 = []
df_170 = []
df_190 = []
df_210 = []
df = df[~np.isnan(df.r_med_photogeo)]

In [ ]:
coords = np.array([df.ra.values, df.dec.values, np.ones_like(df.ra.values), df.pmra.values, df.pmdec.values]).T
uncerts = np.zeros_like(coords)

# Convert from ICRS to Galactic. In this analysis we do not use the uncertainties.
gc, __, __ = coordTransform_matrix.transformIcrsToGal(coords,uncerts)

l = gc[:,0]
l[l<0] = 360.+l[l<0]

df['l'] = l[:]
df['b'] = gc[:,1]
df['mul'] = gc[:,3]
df['mub'] = gc[:,4]

maskpc2kms = 4.7403885

df['vl_true'] = maskpc2kms*df['mul'].values*df['r_med_photogeo'].values
df['vb_true'] = maskpc2kms*df['mub'].values*df['r_med_photogeo'].values

l = np.deg2rad(df['l'].values)
b = np.deg2rad(df['b'].values)

### Estimate velocities assuming zero Galactocentric radial velocity

In [ ]:
Usun,Vsun,Wsun = 11.1, 248.5, 7.25
Rsun = 8.178
zsun = 0.020
Vc = 236.
# Heliocentric coordinates
# X is towards l=0
# Y is towards l=90
# Z is towards north pole
sl,cl,sb,cb = np.sin(l),np.cos(l),np.sin(b),np.cos(b)

df['Xhelio'] = cl*cb*df['r_med_photogeo']
df['Yhelio'] = sl*cb*df['r_med_photogeo']
df['Zhelio'] = sb*df['r_med_photogeo']

df['VXhelio_no_LOS'] =  df['vl_true']*(-sl) + df['vb_true']*(-cl*sb) # missing +vlos*cl*cb
df['VYhelio_no_LOS'] =  df['vl_true']*(cl) + df['vb_true']*(-sl*sb)  # missing +vlos*sl*cb
df['VZhelio_no_LOS'] =  df['vb_true']*(cb)                           # missing +vlos*sb

# Now flip my system like a lunatic

df['Xgal'] = Rsun-df['Xhelio']
df['Ygal'] = -df['Yhelio']
df['Zgal'] = df['Zhelio']+zsun
df['VXgal_no_LOS'] = -(df['VXhelio_no_LOS'] + Usun)
df['VYgal_no_LOS'] = -(df['VYhelio_no_LOS'] + Vsun)
df['VZgal_no_LOS'] =  (df['VZhelio_no_LOS'] + Wsun)
df['Rgal'] = np.sqrt(df['Xgal']**2 + df['Ygal']**2)

# It turns out that the XY plane in these coordinates is ~parallel to the Galactic plane, 
# so no further transformation is required

df['Vlos_guess'] = ((df['Xgal']*df['VXgal_no_LOS'] + df['Ygal']*df['VYgal_no_LOS'])
                     /(df['Xgal']*cl*cb+df['Ygal']*sl*cb))

df['Vphi_guess'] =  df['Rgal']*(-sl*df['VXgal_no_LOS']+cl*df['VYgal_no_LOS'])/(df['Xgal']*cl+df['Ygal']*sl)

df['Vz_guess'] = df['VZgal_no_LOS'] + df['Vlos_guess']*sb
df['R_guide'] = np.abs(df['Rgal']*df['Vphi_guess'])/Vc

In [ ]:
# Save memory & time by taking the needed subset of columns 
df = df[['source_id', 'phot_g_mean_mag', 'bp_rp', 'l', 'b', 'r_med_photogeo',
         'Xgal', 'Ygal', 'Zgal','Rgal', 'Vphi_guess', 'Vz_guess']]

In [ ]:

if not os.path.exists(datadir + 'BayesStarExtinction.h5'):
    import dustmaps
    import astropy.units as u
    from astropy.coordinates import SkyCoord
    from dustmaps.bayestar import BayestarQuery
    bayestar = BayestarQuery(max_samples=2, version='bayestar2017')

    
    coords = SkyCoord(df.l.values*u.degree,df.b.values*u.degree,
                      df.r_med_photogeo.values*u.kpc,frame='galactic')
    ebv = bayestar(coords, mode='median')

    coords = SkyCoord(df.l.values*u.degree,df.b.values*u.degree,df.r_med_photogeo.values*u.kpc,frame='galactic')
    ebv = bayestar(coords, mode='median')
    
    # Values from Sanders & Das
    AG = 2.294*ebv
    EBPRP = (3.046-1.737)*ebv

    df_ext = pd.DataFrame({'source_id' : df.source_id.values, 'ebv' : ebv, 'AG' : AG, 'EBPRP' : EBPRP})
    df_ext.to_hdf(datadir + 'BayesStarExtinction.h5',key='table',mode='w',index=False)
else:
    df_ext = pd.read_hdf(datadir + 'BayesStarExtinction.h5',key='table')


In [ ]:
df = pd.merge(df,df_ext,on='source_id')
df_ext = []
mask = (df.bp_rp-df.EBPRP < 0.5) & (df.phot_g_mean_mag-df.AG-5*np.log10(df.r_med_photogeo/0.01) < 3.)
dfBlue = df[mask]

In [ ]:
print(f'Total {len(df)} stars. Blue sample {len(dfBlue)} stars')


# Lz-Vz plots

In [ ]:
nLz,nVZ = 51,51
Lzbins = np.linspace(-3490,-2010,nLz)
VZbins = np.linspace(-50,50,nVZ)


VZbinwidth = VZbins[1]-VZbins[0]

def makeRgVzPlotFinal_v4(df_in, lmin0, grid_on=False, circleSA=False, fixed_stdVZ=None) :
    '''Function which makes plots of Vz as a function of Lz (and Rg), split in Z
    
    Plots returned for 50 degrees in l (10 degrees per column), divided into panels
    for Z>0 and Z<0.
    
    Note that the colour is effectively mapping P(Vz|Lz), not P(Vz,Lz).
    
    Colour scaling has vmax = 0.03 * 20 / std(Vz), where std(Vz) can be given as fixed value

    Parameters
    ----------
    df_in: pandas DataFrame
        Contains information about all the stars you want to use
    lmin0: float
        lower limit of Galactic longitude
    grid_on: bool (default False)
        If true, each plot has a grid
    circleSA bool (default False)
        If True a circle is drawn in one panel around a feature near Lz = -2350, VZ =-20
    fixed_stdVZ: float (default None)
        If not None, scales vmax of colour map.
    Returns
    -------
    im: image from matplotlib imshow
        Used for colorbars
    fig: matplotlib figure
    
    ax: 2 by 5 matplotlib axes
        
    '''
    
    fig, ax = plt.subplots(2,5,figsize=(13,4.2)) # for double-column plots
 
    for ii in range(10) :
        row = ii//5
        itmp = ii%5
        lmin = lmin0+10*itmp
        lmax = lmin0+10*(itmp+1)

        maskl = ((df_in['l']>lmin) &(df_in['l']<lmax) 
            & (np.absolute(df_in['Vz_guess'])<50.))
        maskZ = df_in['Zgal']>0
        if (ii>=5) : maskZ = ~maskZ 
        mask = maskl & maskZ
        df_tmp = df_in[mask]
        
        Lz_tmp = (df_tmp['Rgal'].values)*df_tmp['Vphi_guess'].values
        H,xedge,yedge = np.histogram2d(Lz_tmp,
                                       df_tmp['Vz_guess'].values,
                                       bins=[Lzbins,VZbins])

        H1,x1 = np.histogram(Lz_tmp, Lzbins)
        for i in range(len(H1)) :
            H[i,:] /= H1[i]*VZbinwidth
        
        if fixed_stdVZ is not None:
            vmax = 0.03* 20./fixed_stdVZ # ad hoc rescaling
        else: 
            vmax = 0.03* 20./np.std(df_tmp['Vz_guess'].values) # ad hoc rescaling

        im = ax[row,itmp].imshow(H.T,origin='lower',extent=[Lzbins[0],Lzbins[-1],-50,50],
                                 aspect=10,vmax=vmax,cmap=mycolormap)
        if circleSA and ii==0 :
            ax[row,itmp].add_artist(mpl.patches.Ellipse(xy=(-2350,-20), width=400, height=40,fill=False,color='green'))
        
        ax[row,itmp].invert_xaxis()
        if grid_on:  ax[row,itmp].grid(True,which='major',axis='x',ls=':',color='k')
        secax1 = ax[row,itmp].secondary_xaxis('top', functions=(lambda x: -x/Vc, lambda x: -x*Vc))
        ax[row,itmp].tick_params(axis='x', which='both', top=False) 
        if row==0:
            secax1.set_xlabel(RGlabel,labelpad=5)
            ax[row,itmp].tick_params(labelbottom=False)
        else:
            secax1.tick_params(labeltop=False)
            ax[row,itmp].set_xlabel(LZlabel)
        if itmp==0:
            ax[row,itmp].set_ylabel(VZlabel)
            ax[row,itmp].tick_params(axis='y', which='major', pad=5)
        else:
            ax[row,itmp].tick_params(labelleft=False)
            
        if (ii<5) :
            ax[row,itmp].set_title(r'$%d^\circ<\ell<%d^\circ$' % (lmin,lmax), fontsize='x-large')
            ax[row,itmp].text(0.9, 0.1, '$Z>0$', fontsize='large', horizontalalignment='right',
                    verticalalignment='bottom', transform=ax[row,itmp].transAxes)
        else :
            ax[row,itmp].text(0.9, 0.1, '$Z<0$', fontsize='large', horizontalalignment='right',
                    verticalalignment='bottom', transform=ax[row,itmp].transAxes)
        
        ii += 1
    #fig.colorbar(im, ax=ax.ravel().tolist())
    return im, fig, ax

## Make and save Vz Lz plots

In [ ]:

plt.rcParams['font.size'] = 12
im, fig, ax = makeRgVzPlotFinal_v4(df,130, grid_on=True, fixed_stdVZ=15.)
plt.tight_layout()

fig.colorbar(im, ax=ax.ravel().tolist(),fraction=0.1,pad=0.02,label=r'$P(V_Z^*)$ [km$^{-1}\,$s]',shrink=0.9)
plt.savefig(plotdir + 'RgVz_l10_130180.png',  format='png', dpi=100,bbox_inches='tight')
plt.savefig(plotdir + 'RgVz_l10_130180.pdf',  format='pdf', dpi=300,bbox_inches='tight')
plt.show()

im, fig, ax = makeRgVzPlotFinal_v4(df,180,grid_on=True,circleSA=True, fixed_stdVZ=15.)
plt.tight_layout()
fig.colorbar(im, ax=ax.ravel().tolist(),fraction=0.1,pad=0.02,label=r'$P(V_Z^*)$ [km$^{-1}\,$s]',shrink=0.9)
plt.savefig(plotdir + 'RgVz_l10_180230.png',  format='png', dpi=100,bbox_inches='tight')
plt.savefig(plotdir + 'RgVz_l10_180230.pdf',  format='pdf', dpi=300,bbox_inches='tight')
plt.show()

### Blue sample: Make and save Vz Lz plots

In [ ]:

plt.rcParams['font.size'] = 12
im, fig, ax = makeRgVzPlotFinal_v4(dfBlue,130, grid_on=True, fixed_stdVZ=10.)
plt.tight_layout()
fig.colorbar(im, ax=ax.ravel().tolist(),fraction=0.1,pad=0.02,label=r'$P(V_Z^*)$ [km$^{-1}\,$s]',shrink=0.9)
plt.savefig(plotdir + 'RgVz_Blue_l10_130180.png',  format='png', dpi=100,bbox_inches='tight')
plt.savefig(plotdir + 'RgVz_Blue_l10_130180.pdf',  format='pdf', dpi=300,bbox_inches='tight')
plt.show()


plt.rcParams['font.size'] = 12
im, fig, ax = makeRgVzPlotFinal_v4(dfBlue,180, grid_on=True, circleSA=True, fixed_stdVZ=10.)
plt.tight_layout()
fig.colorbar(im, ax=ax.ravel().tolist(),fraction=0.1,pad=0.02,label=r'$P(V_Z^*)$ [km$^{-1}\,$s]',shrink=0.9)
plt.savefig(plotdir + 'RgVz_Blue_l10_180230.png',  format='png', dpi=100,bbox_inches='tight')
plt.savefig(plotdir + 'RgVz_Blue_l10_180230.pdf',  format='pdf', dpi=300,bbox_inches='tight')
plt.show()

# Show HR diagram

In [ ]:
plt.rcParams['font.size'] = 13

cbarticks = [0,1000,2500,5000,10000,25000,50000]

fig, ax = plt.subplots(figsize=(6,4.5))
plt.hist2d(df.bp_rp.values-df.EBPRP,
           df.phot_g_mean_mag.values-df.AG-5*np.log10(df.r_med_photogeo.values/0.01),[140,180],
           cmap=mycolormap,
           range=[[-0.5,3],[-6,12]],norm=matplotlib.colors.PowerNorm(0.5,vmax=50000))
plt.gca().invert_yaxis()

cax = plt.colorbar(ticks=cbarticks)
cax.ax.minorticks_off()

# Selection
plt.hlines(3,-0.5,0.5,colors='k',lw=1)
plt.vlines(0.5,-6,3,colors='k',lw=1)

# Reddening vector
plt.quiver(2.5,-4,(3.046-1.737),2.294,angles='xy')

plt.xlabel('$(G_{BP}-G_{RP})_0$')
plt.ylabel('$M_G$')
plt.tight_layout()
ax.set_rasterized(True)
plt.savefig(plotdir + 'HRdiagram.png',  format='png', dpi=100,bbox_inches='tight')
plt.savefig(plotdir + 'HRdiagram.pdf',  format='pdf', dpi=300, 
            bbox_inches='tight')
plt.show()

# Vphi-Vz plots


In [ ]:
vlmin,vlmax = 180,275 #-275,-180
vbmin,vbmax = -40,40

plt.rcParams['font.size'] = 12

def histogramvlvbMasked(df_in,nbins=100) :
    '''Makes a 2d histogram of Vphi vs Vz for input pandas dataframe
    
    Limits given above
    '''
    
    H, xedges, yedges = np.histogram2d(-df_in.Vphi_guess.values,df_in.Vz_guess.values,
                                       [np.linspace(vlmin,vlmax,nbins),np.linspace(vbmin,vbmax,nbins)],
                                       density=True)
    return H

def makeVphiVZFullPlot(df_in, Zdivide=0,Rmin0=11.,fig_x=13.,fig_y=19., grid_on=True) :
    '''
    
    
    '''
    # Number of l bins, z bins, radius bins
    nl,nz,nc=5,2,6
    fig, ax = plt.subplots(nl*nz,nc,figsize=(fig_x,fig_y), constrained_layout=True)
    
    # step in radius
    dR = 0.5
    for i in range(nl*nz*nc) :
        # Which axes
        row = i//nc
        col = i%nc
        # which data are used for these axes
        iR = col
        iL = row//nz #i%2
        iZ = row%nz
        Rmin = Rmin0+iR*dR
        Rmax = Rmin0+(iR+1)*dR
        lmin = 130 + 20*iL
        lmax = 130 + 20*(iL+1)
        
        maskL = (df_in.l.values>lmin) & (df_in.l.values<lmax)
        if iZ==0 :
            mask = (df_in.Rgal.values<Rmax) & (df_in.Rgal.values>Rmin) & (df_in.Zgal.values > Zdivide) & maskL
        if iZ==1 :
            mask = (df_in.Rgal.values<Rmax) & (df_in.Rgal.values>Rmin) & (df_in.Zgal.values < Zdivide) & maskL
        df_tmp = df_in[mask]
        
        a = histogramvlvbMasked(df_tmp,50)
        pdfscale = 1e-4
        pdfscale_text = r'${10^4}$ '
        vmin,vmax = 0./pdfscale,0.0008/pdfscale #np.max(a)
        im = ax[row,col].imshow(a.T/pdfscale,origin='lower',extent=[-vlmin,-vlmax,vbmin,vbmax],aspect=1,
                       vmin=vmin,vmax=vmax,cmap='viridis')
        
        if grid_on: 
            # Predefined grid
            ymin,ymax = ax[row,col].get_ylim()
            ax[row,col].vlines([-200,-250],ymin=ymin,ymax=ymax,ls=':',color='white',alpha=0.4)
            xmin,xmax = ax[row,col].get_xlim()
            ax[row,col].hlines([-20,0,20],xmin=xmin,xmax=xmax,ls=':',color='white',alpha=0.4)
        
        # Cross at 2750 kpc km/s
        if Rmin>=10. :
            ax[row,col].plot(-2750/(0.5*(Rmin+Rmax)),0.,'wx')
        
        # Titles
        if(iZ==0) & (iL==0) :
            if (Rmin-int(Rmin)<0.001):
                ax[row,col].set_title(r'$%d<R<%.1f$' % (Rmin,Rmax), fontdict = {'fontsize': 'xx-large'})
            else:
                ax[row,col].set_title(r'$%.1f<R<%d$' % (Rmin,Rmax), fontdict = {'fontsize': 'xx-large'})
        
        if(iZ==0) | (iL<nl-1):
            ax[row,col].tick_params(labelbottom=False)
        if(iZ==0):
            ax[row,col].text(0.9,0.87,r'$Z>0$',transform=ax[row,col].transAxes,horizontalalignment='right',color='w')

        if (iL == nl-1) & (iZ==1) :
            ax[row,col].set_xlabel(Vplabel, size='large')
            ax[row,col].tick_params(axis='x', labelsize='large')
        
        if (iZ==1):
            ax[row,col].text(0.9,0.87,r'$Z<0$',transform=ax[row,col].transAxes,horizontalalignment='right',color='w')
            
        if iR:
            ax[row,col].tick_params(labelleft=False)
        else :
            ax[row,col].tick_params(axis='y', labelsize='large')
            ax[row,col].set_ylabel(VZlabel, size='large')
            
        ax[row,col].tick_params(which='both',color='white')
        
    # Try to draw dividing lines & label longitude bins
    # Surprisingly ad-hoc and backend reliant
    for i in range(nl):
        suptitle = f'${130+i*20}^\circ<\ell<{150+i*20}^\circ$'
        plt.figtext(x=0.0,y=0.9-0.2*i,s=suptitle,ha='right',va='center',rotation=90,fontsize=30, color='blue')
        
        # Guess upper and lower bound and place lines evenly between them
        yLo,yHi=0.022,0.992
        dy = (yHi-yLo)/5.
        for i in range(1,nl):
            fig.add_artist(mpl.lines.Line2D([0, 1], [yLo+dy*i, yLo+dy*i],color='blue'))
    
    return im, fig, ax

In [ ]:

im, fig, ax = makeVphiVZFullPlot(df, Rmin0=10.5,fig_y=18)

plt.savefig(plotdir + 'VphiVz_all.png',  format='png', dpi=100,bbox_inches='tight')
plt.savefig(plotdir + 'VphiVz_all.pdf',  format='pdf', dpi=300,bbox_inches='tight')
plt.show()